In [11]:
print("Hlloward")

Hlloward


In [12]:
!pip install pandas
import pandas as pd
!pip install numpy
import numpy as np

In [57]:
import pandas as pd
import numpy as np
path = "c:\\Users\\esthelle.kuissu\\OneDrive - Exiom Partners\\Documents\\Code_rapport_stage\\Bases sinistres"
dataM = pd.read_csv(path + "\\new_bdd_sinistres_rapports_2024_final.csv", sep=";", encoding="latin1")
dataM.head()

,cle_sin,pkw,age_veh,val,marq,modele2,generation,airbag,co2min,cdval,...,TAUX_HORAIRE_MO,NB_PIECES_REEMPLOI,NB_PIECE_REPAREE,NB_PIECE_REMPLACEE,RE_AVEC_PIECE_REEMPLOI,MT_TOTAL_REPARABLE,MT_MO_REPARABLE,MT_IP_REPARABLE,MT_PIECES_REPARABLE,MT_FRANCHISE
0,241840204C A 001,74.0,3.0,24584.0,PEU,208,2.0,6.0,97.0,V07,...,63.248428,0.0,3.0,3.0,0.0,471.20,598.300,197.175,604.32,300.0
1,241840598C A 001,103.0,3.0,31980.0,NIS,QASHQAI,2.0,6.0,124.0,V10,...,65.000000,0.0,1.5,1.0,0.0,544.80,65.000,125.000,476.80,350.0
2,241840842C A 006,103.0,18.0,35726.0,VWA,TOURAN,1.0,6.0,159.0,V12,...,35.000000,0.0,1.0,2.0,0.0,270.00,70.000,123.500,200.00,175.0
3,241840544C A 002,50.0,11.0,13464.0,PEU,107,1.0,4.0,99.0,V03,...,35.000000,0.0,1.0,5.0,0.0,302.50,227.500,75.000,0.00,0.0
4,241841770C A 003,62.5,10.0,17640.0,HYU,I20,1.0,6.0,114.0,V05,...,56.390659,0.0,3.0,1.0,0.0,0.01,550.025,151.200,0.01,0.0


In [58]:
Y1 = dataM['MT_MO_REPARABLE']

In [59]:
X1 = dataM[['pkw', 'age_veh', 'val', 'generation', 'airbag', 'co2min', 'anc_permis',
       'cylin', 'poidspuiss', 'crm', 'nab50','NBHH_MO_REPARABLE', 'TAUX_HORAIRE_MO']]

In [60]:
print(X1.dtypes)


pkw                  float64
age_veh              float64
val                  float64
generation           float64
airbag               float64
co2min               float64
anc_permis           float64
cylin                float64
poidspuiss           float64
crm                  float64
nab50                float64
NBHH_MO_REPARABLE    float64
TAUX_HORAIRE_MO      float64
dtype: object


In [61]:
#!pip install pyglmnet
!pip install scipy
from pyglmnet import GLM
from scipy import stats

In [62]:
from pyglmnet import GLM
model = GLM(distr='gamma', alpha=1, reg_lambda=0.01)
import numpy as np

In [63]:
X1 = X1.to_numpy()  # Convertir DataFrame en ndarray
Y1 = Y1.to_numpy()  # Convertir Series en ndarray


In [64]:
model.fit(X1, Y1)  # Réessayer l'ajustement du modèle


c:\Users\esthelle.kuissu\OneDrive - Exiom Partners\Documents\Code_rapport_stage\Note_book\.venv\Lib\site-packages\pyglmnet\pyglmnet.py:92: RuntimeWarning: overflow encountered in exp
  mu = np.log1p(np.exp(z))
c:\Users\esthelle.kuissu\OneDrive - Exiom Partners\Documents\Code_rapport_stage\Note_book\.venv\Lib\site-packages\pyglmnet\pyglmnet.py:863: UserWarning: Reached max number of iterations without convergence.
  warnings.warn(


<
Distribution | gamma
alpha | 1.00
max_iter | 1000.00
lambda: 0.01
>

In [65]:
# Predicting the response variable using the fitted model
Y_pred = model.predict(X1)

In [66]:
dataM['Y_pred'] = Y_pred  # Ajouter les prédictions au DataFrame original
# Calculer les résidus 
dataM['residuals'] = dataM['MT_MO_REPARABLE'] - dataM['Y_pred']
# Afficher les 5 premières lignes du DataFrame avec les prédictions et les résidus
print(dataM[['MT_MO_REPARABLE', 'Y_pred', 'residuals']].head(10))


   MT_MO_REPARABLE      Y_pred   residuals
0          598.300         inf        -inf
1           65.000         inf        -inf
2           70.000         inf        -inf
3          227.500  517.094524 -289.594524
4          550.025  677.437458 -127.412458
5          488.025  572.001228  -83.976228
6         1237.000         inf        -inf
7          108.340         inf        -inf
8           62.400         inf        -inf
9          133.000         inf        -inf


In [ ]:
!pip install pyglmnet



In [13]:
!pip install distutils

ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils


In [18]:
pip install setuptools

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install --upgrade setuptools


Note: you may need to restart the kernel to use updated packages.


In [21]:
from setuptools import setup
